#Code by Latong https://www.kaggle.com/latong/food-review-text-summarization

In [ ]:
import nltk
from bs4 import BeautifulSoup
import string
from nltk.tokenize import RegexpTokenizer


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
df = pd.read_csv('../input/cusersmarildownloadssecondcsv/second.csv', delimiter=';', encoding = "utf8", nrows = nRowsRead)
df.dataframeName = 'second.csv'
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')
df.head()

In [ ]:
#We need only three columns Score, Summary & Text. Let 's remove all the other columns.

#df=df[['Study Type','Factors', 'Excerpt', 'Sample Text', 'Matches']]
#df.head()

I wanted to choose another feature though its length must result INTEGER. 

In [ ]:
#calculating length of lists in pandas dataframe column 
#ref. https://stackoverflow.com/questions/41340341/pythonic-way-for-calculating-length-of-lists-in-pandas-dataframe-column
date=df['Date'].str.len()
print(date)

In [ ]:
# let's check the length of summaries, the average length is 20 characters.
df['date length'] = df['Date'].apply(len)
df['date length'].describe()

In [ ]:
import seaborn as sns
sns.boxplot(x='Study Type', y=df['date length'], data=df)

In [ ]:
# let's do the same to the text, the avearage length is 302 characters.
df['study length'] = df['Study'].apply(len)
df['study length'].describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.boxplot(x='Study Type', y=df['study length'], data=df)
plt.ylim(0, 900)

In [ ]:
# Let 's keep the max length of Date to 30 characters and Study to 300 characters. 
df= df[df['date length'] <=30]
df= df[df['study length'] <=300]

len(df)
df.head()

In [ ]:
df=df[['Date', 'Study Type','Study']]
df.head()
#len(df) the length of updated dataset is 239868

In [ ]:
df = df.reset_index(drop=True)# we need to reindex the dataset after removed some rows
df.head()

In [ ]:
import requests
from bs4 import BeautifulSoup 

def preprocess_text(text):
    """ Apply any preprocessing methods"""
    text = BeautifulSoup(text).get_text()
    text = text.lower()
    text = text.replace('[^\w\s]','')
    return text

df["Study"] = df.Study.apply(preprocess_text)
df["Date"] = df.Date.apply(preprocess_text)

In [ ]:
field_length = df.Study.astype(str).map(len)
print()
print("###This is the longest text###")
print (df.loc[field_length.idxmax(), 'Study'])
print()
print("###This is the shortest text###")
print()
print (df.loc[field_length.idxmin(), 'Study'])

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
#not is in the stopword list. W/O adding whitelist, the summary would change from "do not recommend" to "recommend". This solution was borrowed from bertcarremans https://github.com/bertcarremans/TwitterUSAirlineSentiment
whitelist = ["n't", "not", "no"]

df['Study_after_removed_stopwords'] = df['Study'].apply(lambda x: ' '.join([word for word in x.split() if word in whitelist or word not in (stop)]))
print()
print('###Study after removed stopwords###'+'\n'+df['Study_after_removed_stopwords'][1])
print()
print('###Study before removed stopwords###'+'\n'+ df['Study'][1])
print()
df['Date_after_removed_stopwords'] = df['Date'].apply(lambda x: ' '.join([word for word in x.split() if word in whitelist or word not in (stop)]))
print('###Date after removed stopwords###'+ '\n'+df['Date_after_removed_stopwords'][1])
print()
print('###Date before removed stopwords###'+'\n'+df['Date'][1])

In [ ]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [ ]:
#This two blocks of code was refered from https://www.kaggle.com/currie32/summarizing-text-with-amazon-reviews
def clean_text(text):

    # Replace contractions with longer forms in the above list
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)

In [ ]:

clean_studies = []
for study in df.Study:
    clean_studies.append(clean_text(study))
print("Studies are complete.")

print(len(clean_studies))


clean_texts = []
for text in df.Date:
    clean_texts.append(clean_text(text))
print("Texts are complete.")
print(len(clean_texts))
clean_text

###According to this link https://stackoverflow.com/questions/54891464/is-it-better-to-keras-fit-to-text-on-the-entire-x-data-or-just-the-train-data
and other papers, I will do the tokenization to only train dataset. As I don't have enough memory, I will first export the cleaned dataset into csv format; 
Second subsample the dataset and split the toy dataset into train and test sets; Third, tokenize the train set. 


In [ ]:
df.head()

In [ ]:
r'/amazon_clean

In [ ]:
df.to_csv(r'/cusersmarildownloadssecondcsv_clean.csv',index=False)
len(df)

The value of frac was frac=0.001  That resulted in 0 and df.tail was empty. So I changed to 0.100

In [ ]:
df1=df.sample(frac=0.100, replace=True, random_state=1)
len(df1)

In [ ]:
df1 = df1.reset_index(drop=True)# we need to reindex the dataset after removed some rows

In [ ]:
df1.tail()

In [ ]:
df2=df1[['Study','Date']]
df2.head()
docs=df2.apply(lambda x: " ".join(x), axis=1)
docs.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
docs=df2.apply(lambda x: " ".join(x), axis=1).tolist()

cv=CountVectorizer(max_df=0.85,max_features=10000)
word_count_vector=cv.fit_transform(docs)
 
list(cv.vocabulary_.keys())[:10]
 

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
 
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

In [ ]:
word_count_vector.shape

In [ ]:
"""
https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.XYtStuczbOQ
need the term frequency (term count) vectors for different tasks, use Tfidftransformer.
need to compute tf-idf scores on documents within“training” dataset, use Tfidfvectorizer
need to compute tf-idf scores on documents outside “training” dataset, use either one, both will work.

"""

# print idf values
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])
 
# sort ascending
df_idf.sort_values(by=['idf_weights'])

In [ ]:
df3=df[['Study','Date']]

df3.tail()

In February, we were  "simulating the infected population and spread of Covid-19". Curves, curves, and  more curves from the whole World.

In [ ]:
#https://github.com/kavgan/nlp-in-practice/blob/master/tf-idf/Keyword%20Extraction%20with%20TF-IDF%20and%20SKlearn.ipynb
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

In [ ]:
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [ ]:
# you only needs to do this once, this is a mapping of index to 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
feature_names=cv.get_feature_names()
 
# get the document that we want to extract keywords from
doc=df['Date'][8]
 
#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))
 
#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())
 
#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)
 
# now print the results
print("\n=====Doc=====")
print(doc)
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])
    
print("\n===original study===")
print(df['Study'][8])
 


#I have no clue if I made anything wrong. Probably, yes. Though the Programm didn't return any error.

#Let me know if you read what I should correct. OK?

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#eb3434','#eb3446','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Be patient. Marília Prata, @mpwolke was Here' )